### Librerías

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

## Librerías
import numpy as np
import seaborn as sns
import sys
import cv2 as cv2
import glob
from PIL import Image
import tensorflow as tf
import sys
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import gc
import keras
from sklearn.model_selection import train_test_split
import os


from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import umap
from sklearn.preprocessing import StandardScaler
64
print(tf.__version__)

2023-10-10 03:26:28.885346: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-10 03:26:28.934109: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.8/dist-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mo

2.12.0


In [2]:
## Uso de GPU
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

## Datos

In [3]:
## Obtener todas las rutas de las imágenes en el dataset
rutas_dataset  = glob.glob('/home/willy98/archivos/Datasets/Micrografías_segmentadas/secas/*/*/*/*.jpg')

## Separar rutas
mask_list = []
image_list = []
for ruta in rutas_dataset:
    ruta_list = ruta.split('/')
    clase = ruta_list[7]
    number = ruta_list[8]
    tipo = ruta_list[9]
    name = ruta_list[10].split('.')[0]
    
    if tipo == 'train':
        image_list.append([name,ruta,clase,number])
    elif tipo == 'mask_bin':
        mask_list.append([name,ruta,clase,number])
    else:
        None

mask_pd = pd.DataFrame(mask_list,columns=['name','route_mask','class','number']).sort_values(by='name')
image_pd = pd.DataFrame(image_list,columns=['name','route_image','class','number']).sort_values(by='name')

routes_dataset = image_pd.merge(mask_pd,on=['name','class','number'],how='left')

routes_dataset.dropna(inplace=True)

routes_dataset = routes_dataset.sample(frac = 1)

print('Imágenes secas:', np.shape(routes_dataset)[0])
print('MF1:', '- secas: ', np.shape(routes_dataset[routes_dataset['class']=='MF1'])[0])
print('MF2:', '- secas: ', np.shape(routes_dataset[routes_dataset['class']=='MF2'])[0])
print('MF3:', '- secas: ', np.shape(routes_dataset[routes_dataset['class']=='MF3'])[0])

char_to_num = {'MF1':'0','MF2':'1','MF3':'2'}

routes_dataset['class'].replace(char_to_num,inplace=True)

routes_dataset

Imágenes secas: 927
MF1: - secas:  378
MF2: - secas:  288
MF3: - secas:  261


,name,route_image,class,number,route_mask
258,IMG_20230707_173506_jpg,/home/willy98/archivos/Datasets/Micrografías_s...,0,4,/home/willy98/archivos/Datasets/Micrografías_s...
464,IMG_20230707_191821_jpg,/home/willy98/archivos/Datasets/Micrografías_s...,0,7,/home/willy98/archivos/Datasets/Micrografías_s...
235,IMG_20230707_171027_jpg,/home/willy98/archivos/Datasets/Micrografías_s...,0,3,/home/willy98/archivos/Datasets/Micrografías_s...
376,IMG_20230707_184632_jpg,/home/willy98/archivos/Datasets/Micrografías_s...,0,6,/home/willy98/archivos/Datasets/Micrografías_s...
108,IMG_20230707_161859_jpg,/home/willy98/archivos/Datasets/Micrografías_s...,0,1,/home/willy98/archivos/Datasets/Micrografías_s...
...,...,...,...,...,...
841,IMG_20230711_212837_jpg,/home/willy98/archivos/Datasets/Micrografías_s...,2,7,/home/willy98/archivos/Datasets/Micrografías_s...
827,IMG_20230711_212453_jpg,/home/willy98/archivos/Datasets/Micrografías_s...,2,7,/home/willy98/archivos/Datasets/Micrografías_s...
790,IMG_20230711_211442_jpg,/home/willy98/archivos/Datasets/Micrografías_s...,2,2,/home/willy98/archivos/Datasets/Micrografías_s...
218,IMG_20230707_170821_jpg,/home/willy98/archivos/Datasets/Micrografías_s...,0,3,/home/willy98/archivos/Datasets/Micrografías_s...


In [4]:
## Establecer el porcentaje de separación
prct_train = 0.8

thold1 = int(np.ceil(prct_train * len(routes_dataset[routes_dataset['class']=='0'])))
thold2 = int(np.ceil(prct_train * len(routes_dataset[routes_dataset['class']=='1'])))
thold3 = int(np.ceil(prct_train * len(routes_dataset[routes_dataset['class']=='2'])))

## Separar por clases para entrenamiento
train_cu1 = routes_dataset[routes_dataset['class'] == '0'][0:thold1]
train_cu2 = routes_dataset[routes_dataset['class'] == '1'][0:thold2]
train_cu3 = routes_dataset[routes_dataset['class'] == '2'][0:thold3]

## tomar las demás imágenes para test
test_cu1 = routes_dataset[routes_dataset['class'] == '0'][thold1:]
test_cu2 = routes_dataset[routes_dataset['class'] == '1'][thold2:]
test_cu3 = routes_dataset[routes_dataset['class'] == '2'][thold3:]

## Cantidad de muestras por clase
print('Clase MF1: train: ',len(train_cu1),', test: ',len(test_cu1))
print('Clase MF2: train: ',len(train_cu2),', test: ',len(test_cu2))
print('Clase MF3: train: ',len(train_cu3),', test: ',len(test_cu3))

Clase MF1: train:  303 , test:  75
Clase MF2: train:  231 , test:  57
Clase MF3: train:  209 , test:  52


In [5]:
### Definir datasets de training y test
# Todas las clases
#train_df =  pd.concat([pd.concat([train_cu1,train_cu2]),train_cu3])
#test_df = pd.concat([pd.concat([test_cu1,test_cu2]),test_cu3])

# solo una clase
train_df = train_cu1
test_df = test_cu1

train_df

,name,route_image,class,number,route_mask
258,IMG_20230707_173506_jpg,/home/willy98/archivos/Datasets/Micrografías_s...,0,4,/home/willy98/archivos/Datasets/Micrografías_s...
464,IMG_20230707_191821_jpg,/home/willy98/archivos/Datasets/Micrografías_s...,0,7,/home/willy98/archivos/Datasets/Micrografías_s...
235,IMG_20230707_171027_jpg,/home/willy98/archivos/Datasets/Micrografías_s...,0,3,/home/willy98/archivos/Datasets/Micrografías_s...
376,IMG_20230707_184632_jpg,/home/willy98/archivos/Datasets/Micrografías_s...,0,6,/home/willy98/archivos/Datasets/Micrografías_s...
108,IMG_20230707_161859_jpg,/home/willy98/archivos/Datasets/Micrografías_s...,0,1,/home/willy98/archivos/Datasets/Micrografías_s...
...,...,...,...,...,...
202,IMG_20230707_164706_jpg,/home/willy98/archivos/Datasets/Micrografías_s...,0,2,/home/willy98/archivos/Datasets/Micrografías_s...
318,IMG_20230707_180420_jpg,/home/willy98/archivos/Datasets/Micrografías_s...,0,5,/home/willy98/archivos/Datasets/Micrografías_s...
425,IMG_20230707_191248_jpg,/home/willy98/archivos/Datasets/Micrografías_s...,0,7,/home/willy98/archivos/Datasets/Micrografías_s...
272,IMG_20230707_173702_jpg,/home/willy98/archivos/Datasets/Micrografías_s...,0,4,/home/willy98/archivos/Datasets/Micrografías_s...


In [18]:
## Crear objetos transformadores
## Para entrenamiento
SEED = 100

img_generator_images = tf.keras.preprocessing.image.ImageDataGenerator(  
    rescale=1./255,      
    horizontal_flip=True,
    rotation_range=90,
    vertical_flip=True,
    validation_split=0.2
)

img_generator_mask = tf.keras.preprocessing.image.ImageDataGenerator(  
    rescale=1./255,      
    horizontal_flip=True,
    rotation_range=90,
    vertical_flip=True,
    validation_split=0.2
)

## Crear objeto generador de imágenes a partir del objeto transformator
## Pra entrenamiento
train_generator_IMAGES = img_generator_images.flow_from_dataframe(
    dataframe=train_df,
    #directory="./train/",
    x_col="route_image",
    y_col="class",
    labels = ['MF1'],
    #labels = ['MF2'],
    subset="training",
    batch_size=32,
    seed=SEED,
    shuffle=True,
    class_mode="sparse" ,
    validation_split=0.2,
    color_mode ="grayscale",
    target_size=(128,128))

train_generator_MASK = img_generator_mask.flow_from_dataframe(
    dataframe=train_df,
    #directory="./train/",
    x_col="route_mask",
    y_col="class",
    labels = ['MF1'],
    #labels = ['MF2'],
    subset="training",
    batch_size=32,
    seed=SEED,
    shuffle=True,
    class_mode="sparse",
    validation_split=0.2,
    color_mode ="grayscale",
    target_size=(128,128))

Found 243 validated image filenames belonging to 1 classes.
Found 243 validated image filenames belonging to 1 classes.


## Modelo

In [19]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

class VAE(keras.Model):
    def __init__(self,encoder,decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        ##Variables que guardan paso a paso las pérdidas
        self.kl_loss_tracker = keras.metrics.Mean(name='kl_loss')        
        self.reconstruction_loss_tracker = keras.metrics.Mean(name='reconstruction_loss')
        self.total_loss_tracker = keras.metrics.Mean(name='total_loss')
        
    @property
    def metrics(self):
        return[
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    
    def train_step(self,data):
        with tf.GradientTape() as tape:
            ## Generar embebidos
            z_mean, z_log_var, z = self.encoder(data[0])
            
            ## Reconstruir embebidos
            reconstruction = self.decoder(z)
            
            ## Normalizar reconstrucciones
            reconstruction /= tf.reduce_max(reconstruction)
            
            ## Calcular la diferencia binaria entre las imágenes de entrada y las reconstrucciones
            
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data[0], reconstruction), axis=(1, 2)
                )
            )         
            
            '''
            reconstruction_loss =tf.reduce_mean(
                tf.reduce_sum( tf.keras.losses.MeanSquaredError()(data, reconstruction)
                             )
            )
            '''
            
            ## Pérdida de kullback leibler
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))            
        
            ## Pérdida total
            total_loss = reconstruction_loss + kl_loss     
            
        ## Calcular los gradientes 
        grads = tape.gradient(total_loss, self.trainable_weights)       

        ## Guardar reconstrucciones en una variable propia del modelo
        self.reconstruction = reconstruction

        ## Calcular gradientes entre los pesos            
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        ## Actualizar la variable de pérdida total
        self.total_loss_tracker.update_state(total_loss)

        ## Actualizar la variable de pérdida de reconstrucción
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)       

        ## Actualizar la variable de pérdida por kull-back
        self.kl_loss_tracker.update_state(kl_loss)       

        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(), 
            'kl_loss':self.kl_loss_tracker.result(),
        }

In [20]:
### Encoder
latent_dim = 128

encoder_inputs = keras.Input(shape=(128, 128, 1))

x = tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same',name='layer_E1')(encoder_inputs)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same',name='layer_E3')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same',name='layer_E2')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same',name='layer_E4')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same',name='layer_E6')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same',name='layer_E7')(x)
x = layers.Flatten()(x)



z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(inputs=encoder_inputs, outputs=[z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 layer_E1 (Conv2D)              (None, 128, 128, 10  10240       ['input_5[0][0]']                
                                24)                                                               
                                                                                                  
 max_pooling2d_10 (MaxPooling2D  (None, 64, 64, 1024  0          ['layer_E1[0][0]']               
 )                              )                                                           

In [21]:
### Decoder
## Entrada Z
latent_inputs = keras.Input(shape=(latent_dim,))

##dimensiones
ndim = 8
nfilts = 128

## anti-flatten de la última capa convolucional
x = layers.Dense(ndim * ndim * nfilts, activation="relu")(latent_inputs)

## Reshape para reconstruir la última convolucional
x = layers.Reshape((ndim,ndim, nfilts))(x)

## Capas convolucionales
x = tf.keras.layers.Conv2D(nfilts, (3, 3), activation='relu', padding='same',name='layer_D1')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(nfilts/2, (3, 3), activation='relu', padding='same',name='layer_D2')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(nfilts/4, (3, 3), activation='relu', padding='same',name='layer_D4')(x)
x = tf.keras.layers.UpSampling2D((4, 4))(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(nfilts/8, (3, 3), activation='relu', padding='same',name='layer_D6')(x)

decoder_outputs = tf.keras.layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same',name='layer_D7')(x)

## Construcción del decoder
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 128)]             0         
                                                                 
 dense_2 (Dense)             (None, 8192)              1056768   
                                                                 
 reshape_2 (Reshape)         (None, 8, 8, 128)         0         
                                                                 
 layer_D1 (Conv2D)           (None, 8, 8, 128)         147584    
                                                                 
 up_sampling2d_6 (UpSampling  (None, 16, 16, 128)      0         
 2D)                                                             
                                                                 
 batch_normalization_21 (Bat  (None, 16, 16, 128)      512       
 chNormalization)                                          

In [ ]:
vae = VAE(encoder, decoder)
opt = tf.optimizers.Adam(learning_rate = 0.00001)
#nll = lambda x , rv_x: -rv_x.log_prob(x)
vae.compile(optimizer=opt)


vae.fit(train_generator_IMAGES, epochs=4000, 
                batch_size=16,
                         callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=100)])

Epoch 1/4000


2023-10-10 03:33:51.574916: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-10-10 03:34:02.716302: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.80GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-10-10 03:34:02.716357: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.80GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-10-10 03:34:07.957203: W tensorflow/tsl/framework/bfc_allocator

8/8 [==============================] - 26s 1s/step - loss: 34519.9041 - reconstruction_loss: 34405.7031 - kl_loss: 0.0235
Epoch 2/4000
8/8 [==============================] - 4s 493ms/step - loss: 33543.6944 - reconstruction_loss: 33572.7500 - kl_loss: 0.0361
Epoch 3/4000
8/8 [==============================] - 4s 499ms/step - loss: 33608.7543 - reconstruction_loss: 33616.1406 - kl_loss: 0.0398
Epoch 4/4000
8/8 [==============================] - 4s 515ms/step - loss: 32655.0490 - reconstruction_loss: 32901.6992 - kl_loss: 0.0363
Epoch 5/4000
8/8 [==============================] - 4s 509ms/step - loss: 32819.3804 - reconstruction_loss: 32308.9766 - kl_loss: 0.0496
Epoch 6/4000
8/8 [==============================] - 4s 494ms/step - loss: 31842.5911 - reconstruction_loss: 31578.1250 - kl_loss: 0.0746
Epoch 7/4000
8/8 [==============================] - 4s 516ms/step - loss: 31214.1764 - reconstruction_loss: 30958.2285 - kl_loss: 0.1266
Epoch 8/4000
8/8 [==============================] - 4s 5

8/8 [==============================] - 4s 498ms/step - loss: 11865.2109 - reconstruction_loss: 11159.3398 - kl_loss: 665.0762
Epoch 60/4000
8/8 [==============================] - 5s 544ms/step - loss: 11685.5853 - reconstruction_loss: 11066.5635 - kl_loss: 643.6516
Epoch 61/4000
8/8 [==============================] - 5s 514ms/step - loss: 11791.3363 - reconstruction_loss: 11266.2412 - kl_loss: 520.5743
Epoch 62/4000
8/8 [==============================] - 5s 506ms/step - loss: 11782.6700 - reconstruction_loss: 11208.6357 - kl_loss: 532.9969
Epoch 63/4000
8/8 [==============================] - 4s 511ms/step - loss: 11748.6238 - reconstruction_loss: 11135.1670 - kl_loss: 598.3072
Epoch 64/4000
8/8 [==============================] - 4s 562ms/step - loss: 11676.6424 - reconstruction_loss: 11067.9434 - kl_loss: 585.3838
Epoch 65/4000
8/8 [==============================] - 4s 515ms/step - loss: 11650.9707 - reconstruction_loss: 11076.8291 - kl_loss: 518.5901
Epoch 66/4000
8/8 [===============

8/8 [==============================] - 4s 503ms/step - loss: 11050.9042 - reconstruction_loss: 10808.8516 - kl_loss: 283.5725
Epoch 118/4000
8/8 [==============================] - 5s 554ms/step - loss: 11073.5199 - reconstruction_loss: 10822.1055 - kl_loss: 266.0821
Epoch 119/4000
8/8 [==============================] - 5s 583ms/step - loss: 11126.5544 - reconstruction_loss: 10885.8311 - kl_loss: 244.8295
Epoch 120/4000
8/8 [==============================] - 5s 527ms/step - loss: 11093.8477 - reconstruction_loss: 10810.7012 - kl_loss: 253.3165
Epoch 121/4000
8/8 [==============================] - 4s 536ms/step - loss: 11065.2028 - reconstruction_loss: 10785.9043 - kl_loss: 257.2531
Epoch 122/4000
8/8 [==============================] - 4s 505ms/step - loss: 11057.0642 - reconstruction_loss: 10811.4492 - kl_loss: 243.2366
Epoch 123/4000
8/8 [==============================] - 5s 523ms/step - loss: 11092.8118 - reconstruction_loss: 10839.7959 - kl_loss: 237.2538
Epoch 124/4000
8/8 [========

8/8 [==============================] - 5s 518ms/step - loss: 10850.4533 - reconstruction_loss: 10758.0625 - kl_loss: 146.3952
Epoch 176/4000
8/8 [==============================] - 4s 511ms/step - loss: 10923.2829 - reconstruction_loss: 10790.0742 - kl_loss: 140.3138
Epoch 177/4000
8/8 [==============================] - 4s 481ms/step - loss: 10897.4060 - reconstruction_loss: 10745.8877 - kl_loss: 138.2138
Epoch 178/4000
8/8 [==============================] - 5s 570ms/step - loss: 10877.5371 - reconstruction_loss: 10753.7441 - kl_loss: 139.4097
Epoch 179/4000
8/8 [==============================] - 5s 561ms/step - loss: 10862.9248 - reconstruction_loss: 10740.9238 - kl_loss: 141.1853
Epoch 180/4000
8/8 [==============================] - 5s 517ms/step - loss: 10900.8887 - reconstruction_loss: 10789.6289 - kl_loss: 139.9361
Epoch 181/4000
8/8 [==============================] - 4s 500ms/step - loss: 10874.5694 - reconstruction_loss: 10737.2588 - kl_loss: 143.7048
Epoch 182/4000
8/8 [========

8/8 [==============================] - 5s 530ms/step - loss: 10792.2812 - reconstruction_loss: 10698.8945 - kl_loss: 91.6563
Epoch 234/4000
8/8 [==============================] - 5s 523ms/step - loss: 10802.6420 - reconstruction_loss: 10685.3066 - kl_loss: 101.3161
Epoch 235/4000
8/8 [==============================] - 4s 504ms/step - loss: 10797.2614 - reconstruction_loss: 10694.6826 - kl_loss: 94.7016
Epoch 236/4000
8/8 [==============================] - 4s 512ms/step - loss: 10835.0765 - reconstruction_loss: 10731.1055 - kl_loss: 93.9548
Epoch 237/4000
8/8 [==============================] - 5s 566ms/step - loss: 10856.6451 - reconstruction_loss: 10706.7002 - kl_loss: 111.9420
Epoch 238/4000
8/8 [==============================] - 4s 518ms/step - loss: 10797.9714 - reconstruction_loss: 10709.3262 - kl_loss: 95.9316
Epoch 239/4000
8/8 [==============================] - 4s 536ms/step - loss: 10831.0242 - reconstruction_loss: 10713.7256 - kl_loss: 86.0274
Epoch 240/4000
8/8 [=============

In [ ]:
vae.encoder.save('Models/GVAE_DRY_MF1_128_1_encoder.h5')
vae.decoder.save('Models/GVAE_DRY_MF1_128_1_decoder.h5')